In [23]:
FILE = "../data/osm/NewYork.osm"
with open(FILE, encoding="utf-8") as f:
    data_ny = f.readlines()

In [24]:
import os, math

In [25]:
import os, math
size = os.path.getsize(FILE) / math.pow(1024, 3)
print("{:.3} GB".format(size))

1.21 GB


In [26]:
[x.strip() for x in data_ny[239:246]]

['<node id="42421728" lat="40.7980486" lon="-73.9600434" version="1">',
 '<tag k="highway" v="traffic_signals"/>',
 '<tag k="crossing" v="traffic_signals"/>',
 '</node>',
 '<node id="42421731" lat="40.798645" lon="-73.9614743" version="1">',
 '<tag k="highway" v="traffic_signals"/>',
 '</node>']

In [27]:
data_ny[240]

'\t\t<tag k="highway" v="traffic_signals"/>\n'

In [28]:
data_ny[240].split('"')

['\t\t<tag k=', 'highway', ' v=', 'traffic_signals', '/>\n']

In [29]:
from operator import itemgetter
extract_tag = itemgetter(1, 3)

In [30]:
extract_tag([1, 2, 3, 4])

(2, 4)

In [31]:
tags_ny = [extract_tag(line.split('"')) \
           for line in data_ny \
           if "tag" in line ]

In [32]:
tags_ny[1:100000:10000]

[('name', 'Hell&#39;s Kitchen'),
 ('highway', 'turning_circle'),
 ('highway', 'traffic_signals'),
 ('highway', 'turning_circle'),
 ('name', 'Public School 215'),
 ('gnis:feature_id', '2080342'),
 ('amenity', 'place_of_worship'),
 ('gnis:feature_id', '2054680'),
 ('name', 'FDNY E331 L173'),
 ('cityracks.large', '1')]

In [33]:
from collections import Counter
c = Counter(tags_ny)

In [34]:
c.most_common(5)

[(('building', 'yes'), 1144890),
 (('building', 'garage'), 103371),
 (('tiger:reviewed', 'no'), 82973),
 (('tiger:cfcc', 'A41'), 80459),
 (('highway', 'residential'), 63866)]

In [35]:
DIR = "../data/osm/southamerica"
size = sum(
    os.path.getsize(os.path.join(DIR, f)) \
    for f in os.listdir(DIR)
) / math.pow(1024, 3)
print("{:.3} GB".format(size))

5.22 GB


In [39]:
# Reference:
# https://stackoverflow.com/questions/42339876/error-unicodedecodeerror-utf-8-codec-cant-decode-byte-0xff-in-position-0-in

lines = []
for file in os.listdir(DIR):
    with open(os.path.join(DIR, file), encoding="utf-8", errors='ignore') as f: #add argument errors='ignore' to skip the error
        lines.append(f.readlines())

In [40]:
import dask.bag as db

lst = [1, 2, 3, 4]
bag = db.from_sequence(lst, npartitions=1)
bag

dask.bag<from_sequence, npartitions=1>

In [41]:
bag.map(lambda x: x + 1)

dask.bag<lambda, npartitions=1>

In [42]:
bag.map(lambda x: x + 1).compute()

[2, 3, 4, 5]

In [43]:
bag.fold(lambda acc, e: acc + e, initial=0).compute()

10

In [44]:
bag.sum().compute()

10

In [45]:
bag.fold(lambda acc, e: acc * e, initial=0).compute()

0

In [46]:
bag.foldby(lambda x: x % 2 == 0,
           lambda x, y: x + y,
           initial=0).compute()

[(False, 4), (True, 6)]

In [53]:
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:60247,Cluster Workers: 4 Cores: 4 Memory: 8.59 GB


In [48]:
data = db.read_text("../data/osm/southamerica/southam*")
data

dask.bag<bag-from-delayed, npartitions=51>

In [49]:
task = data.filter(
            lambda x: "tag" in x
        ).foldby(
            lambda x: extract_tag(x.split('"')),
            lambda x, _: x + 1, 0
        ).topk(10, itemgetter(1))
task

dask.bag<topk-aggregate, npartitions=1>

In [50]:
from dask.diagnostics import ProgressBar
with ProgressBar():
    counts = task.compute()

In [51]:
list(counts)

[(('created_by', 'JOSM'), 364794),
 (('source', 'ICRC, DigitalGlobe 120311'), 12258),
 (('source_ref', 'http://earth-info.nga.mil/gns/html/cntyfile/ar.zip'), 5581),
 (('source', 'ISS-004-E-12987, 12988, 12989 Imagery'), 4871),
 (('gns_classification', 'PPL'), 4128),
 (('source', 'Lakewalker plugin'), 2275),
 (('highway', 'traffic_signals'), 2033),
 (('waterway', 'river'), 1754),
 (('gns_classification', 'STM'), 1727),
 (('place', 'isolated_dwelling'), 1662)]